In [1]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import evaluate,train,prepare
from models.SFI_FIM import SFIModel_gating,SFIModel_unified

In [2]:
hparams = {
    'scale':'demo',
    'name':'sfi-fim',
    'batch_size':10,
    'title_size':20,
    'his_size':50,
    'npratio':4,
    'dropout_p':0.2,
    'embedding_dim':300,
    'filter_num':150,
    'epochs':1,
    'metrics':'auc,mean_mrr,ndcg@5,ndcg@10',
    'device':'cuda:1',
    'attrs': ['title'],
    'integration':'harmony',
    'k':10,
    'save_step':0,
    'save_each_epoch':True,
    'train_embedding':True,
    'mode':'train',
    'news_id':False,
    'validate':False,
}

In [3]:
vocab, loaders = prepare(hparams)

[2021-03-12 14:13:18,217] INFO (root) Hyper Parameters are
{'scale': 'demo', 'name': 'sfi-fim', 'batch_size': 10, 'title_size': 20, 'his_size': 50, 'npratio': 4, 'dropout_p': 0.2, 'embedding_dim': 300, 'filter_num': 150, 'epochs': 1, 'metrics': 'auc,mean_mrr,ndcg@5,ndcg@10', 'device': 'cuda:1', 'attrs': ['title'], 'integration': 'harmony', 'k': 10, 'save_step': 0, 'save_each_epoch': True, 'train_embedding': True, 'mode': 'train', 'news_id': False, 'validate': False}
[2021-03-12 14:13:18,218] INFO (root) preparing dataset...
[2021-03-12 14:13:20,441] INFO (torchtext.vocab) Loading vectors from .vector_cache/glove.840B.300d.txt.pt


In [4]:
hparams['select'] = 'gating'
sfiModel_gating = SFIModel_gating(hparams, vocab=vocab).to(hparams['device'])

In [ ]:
train(sfiModel_gating, hparams, loaders, interval=10)

In [ ]:
hparams['select'] = 'unified'
sfiModel_unified = SFIModel_unified(hparams, vocab=vocab).to(hparams['device'])

In [ ]:
train(sfiModel_unified, hparams, loaders, interval=10)

In [ ]:
hparams['select'] = 'unified'
hparams['integration'] = 'gate'
sfiModel_unified = SFIModel_unified(hparams, vocab=vocab).to(hparams['device'])

In [ ]:
train(sfiModel_unified, hparams, loaders, interval=10)